In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

In [2]:
# Read in CSV File
document_check = 'document_check_sample.csv'
facial_report = 'facial_report_sample.csv'

document_df = pd.read_csv(document_check)
facial_df = pd.read_csv(facial_report)


## Understand the attributes of data frames: 
- What are the attributes 
- What are their unique values 

In [3]:
# Understand the attributes
document_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11760 entries, 0 to 11759
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          5880 non-null   float64
 1   user_id                             5880 non-null   object 
 2   result                              5880 non-null   object 
 3   visual_authenticity_result          4991 non-null   object 
 4   image_integrity_result              5880 non-null   object 
 5   face_detection_result               4990 non-null   object 
 6   image_quality_result                5880 non-null   object 
 7   created_at                          5880 non-null   object 
 8   supported_document_result           5869 non-null   object 
 9   conclusive_document_quality_result  3140 non-null   object 
 10  colour_picture_result               3140 non-null   object 
 11  data_validation_result              4739 

In [4]:
# Understand the attributes
facial_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11760 entries, 0 to 11759
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     5880 non-null   float64
 1   user_id                        5880 non-null   object 
 2   result                         5880 non-null   object 
 3   face_comparison_result         5535 non-null   object 
 4   created_at                     5880 non-null   object 
 5   facial_image_integrity_result  5861 non-null   object 
 6   visual_authenticity_result     4991 non-null   object 
 7   properties                     5880 non-null   object 
 8   attempt_id                     5880 non-null   object 
dtypes: float64(1), object(8)
memory usage: 827.0+ KB


## Dropping Nan vales 
- Based on the information above, there are 11760 entries, however there are only 5880 rows with result. Which means we canont make any assumption out of the rows with no result
- Hence, I will decide to drop all those rows

In [5]:
# Dropping the rows with Na values for result
document_df = document_df.dropna(subset=['result']).reset_index(drop=True)
facial_df = facial_df.dropna(subset=['result']).reset_index(drop=True)



## Handling document file

In [6]:
# Remmove unrelevant attributes

document_importantAttr = [
 'result', 
 'visual_authenticity_result',
 'image_integrity_result',
 'face_detection_result',
 'image_quality_result',
 'created_at',
 'supported_document_result',
 'conclusive_document_quality_result',
 'colour_picture_result',
 'data_validation_result',
 'data_consistency_result',
 'data_comparison_result',
 'police_record_result',
 'compromised_document_result',]

In [7]:
# Currently, the date time is object so it will not be sorted correctly. We will convert that to date-time format

document_df['created_at'] = pd.to_datetime(document_df['created_at'], format='%d/%m/%Y %H:%M')
document_df['created_at'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 5880 entries, 0 to 5879
Series name: created_at
Non-Null Count  Dtype         
--------------  -----         
5880 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 46.1 KB


In [8]:
# Since the result recently decreased, sort the dataset by date 'created_at' 

document_filtered = document_df[document_importantAttr]
document_filtered = document_filtered.sort_values(by='created_at').reset_index(drop=True)

In [9]:
# Check the unique values in all attributes 

for attribute in document_importantAttr:
    unique_values = document_filtered[attribute].unique()
    print(f"Unique values for {attribute}:\n{unique_values}\n")


Unique values for result:
['clear' 'consider']

Unique values for visual_authenticity_result:
['clear' 'consider' nan]

Unique values for image_integrity_result:
['clear' 'consider']

Unique values for face_detection_result:
['clear' nan 'consider']

Unique values for image_quality_result:
['clear' 'unidentified']

Unique values for created_at:
['2017-05-23T17:25:00.000000000' '2017-05-23T19:17:00.000000000'
 '2017-05-23T20:49:00.000000000' ... '2017-10-31T21:11:00.000000000'
 '2017-10-31T21:46:00.000000000' '2017-10-31T22:06:00.000000000']

Unique values for supported_document_result:
['clear' nan 'unidentified']

Unique values for conclusive_document_quality_result:
[nan 'clear' 'consider']

Unique values for colour_picture_result:
[nan 'clear' 'consider']

Unique values for data_validation_result:
['clear' nan 'consider']

Unique values for data_consistency_result:
[nan 'clear' 'consider']

Unique values for data_comparison_result:
['clear' 'consider' nan]

Unique values for police_

In [10]:
divide = ['created_at', 'visual_authenticity_result', 'image_integrity_result'  ]

divided_doc = document_filtered[divide]

In [11]:
divided_doc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5880 entries, 0 to 5879
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   created_at                  5880 non-null   datetime64[ns]
 1   visual_authenticity_result  4991 non-null   object        
 2   image_integrity_result      5880 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 137.9+ KB


In [12]:
divided_doc['visual_authenticity_result'].unique()

array(['clear', 'consider', nan], dtype=object)

C:\Users\vuanh\AppData\Local\Temp\ipykernel_3020\2681337610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  divided_doc['date'] = divided_doc['created_at'].dt.date
C:\Users\vuanh\AppData\Local\Temp\ipykernel_3020\2681337610.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  divided_doc['month'] = divided_doc['created_at'].dt.to_period('M')


ValueError: Length of values (159) does not match length of index (5880)